In [1]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML

test = """Poker Hand #OM340571453: PLO ($0.02/$0.05) - 2023/01/03 11:24:11
Table 'PLOYellow2' 6-max Seat #3 is the button
Seat 1: 9cd5e1e6 ($7.98 in chips)
Seat 2: d38f8e13 ($4.34 in chips)
Seat 3: b160657f ($5.67 in chips)
Seat 4: b4180a8e ($8.98 in chips)
Seat 5: Hero ($7.51 in chips)
Seat 6: 4bf5b1f5 ($1.79 in chips)
b4180a8e: posts small blind $0.02
Hero: posts big blind $0.05
*** HOLE CARDS ***
Dealt to 9cd5e1e6 
Dealt to d38f8e13 
Dealt to b160657f 
Dealt to b4180a8e 
Dealt to Hero [Qd 9h 9s Th]
Dealt to 4bf5b1f5 
4bf5b1f5: calls $0.05
9cd5e1e6: calls $0.05
d38f8e13: calls $0.05
b160657f: calls $0.05
b4180a8e: calls $0.03
Hero: raises $0.3 to $0.35
4bf5b1f5: folds
9cd5e1e6: calls $0.3
d38f8e13: calls $0.3
b160657f: folds
b4180a8e: folds
*** FLOP *** [4h 6c 2s]
Hero: checks
9cd5e1e6: bets $1.2
d38f8e13: raises $2.79 to $3.99 and is all-in
Hero: folds
9cd5e1e6: calls $2.79
d38f8e13: shows [6d 7s 5d 8s] (Pair of Sixes)
9cd5e1e6: shows [5s 2h 6h 4s] (Pair of Sixes and Pair of Fours)
*** TURN *** [4h 6c 2s] [3s]
d38f8e13: Chooses to EV Cashout 
*** RIVER *** [4h 6c 2s 3s] [Ts]
d38f8e13: Pays Cashout Risk ($1.43)
*** SHOWDOWN ***
d38f8e13 collected $8.98 from pot
*** SUMMARY ***
Total pot $9.18 | Rake $0.15 | Jackpot $0.05 | Bingo $0 | Fortune $0
Board [4h 6c 2s 3s Ts]
Seat 1: 9cd5e1e6 showed [5s 2h 6h 4s] and lost with Ten High Flush
Seat 2: d38f8e13 showed [6d 7s 5d 8s] and won ($8.98) with Ten High Flush, Cashout Risk ($1.43)
Seat 3: b160657f (button) folded before Flop
Seat 4: b4180a8e (small blind) folded before Flop
Seat 5: Hero (big blind) folded on the Flop
Seat 6: 4bf5b1f5 folded before Flop"""
    

substitutions = {
                     'LEGAL_ISO' : "USD|CNY",      # legal ISO currency codes
                            'LS' : u"\$|\¥|", # legal currency symbols - Euro(cp1252, utf-8)
                           'PLYR': r'\s?(?P<PNAME>.+?)',
                            'CUR': u"(\$|\¥|)",
                          'BRKTS': r'(\(button\) |\(small blind\) |\(big blind\) |\(button blind\) |\(button\) \(small blind\) |\(small blind/button\) |\(button\) \(big blind\) )?',
                    }

re_GameInfo     = re.compile(u"""
          Poker\sHand\s\#[A-Z]{0,2}(?P<HID>[0-9]+):\s+
          (\{.*\}\s+)?((?P<TOUR>((Zoom|Rush)\s)?(Tournament))\s\#                # open paren of tournament info
          (?P<TOURNO>\d+),\s
          # here's how I plan to use LS
          (?P<TOURNAME>.+?)\s
          )?
          # close paren of tournament info
          (?P<GAME>Hold\'em|Hold\'em|ShortDeck|Omaha|PLO|Omaha\sHi/Lo|PLO\-(5|6))\s
          (?P<LIMIT>No\sLimit|Fixed\sLimit|Limit|Pot\sLimit|\(NL\spostflop\))?,?\s*
          (-\s)?
          (?P<SHOOTOUT>Match.*,\s)?
          (Level(?P<LEVEL>[IVXLC\d]+)\s?)?
          \(?                            # open paren of the stakes
          (?P<CURRENCY>%(LS)s|)?
          (ante\s\d+,\s)?
          ((?P<SB>[,.0-9]+)/(%(LS)s)?(?P<BB>[,.0-9]+)|(?P<BUB>[,.0-9]+))
          (?P<CAP>\s-\s[%(LS)s]?(?P<CAPAMT>[,.0-9]+)\sCap\s-\s)?        # Optional Cap part
          \s?(?P<ISO>%(LEGAL_ISO)s)?
          \)                        # close paren of the stakes
          (?P<BLAH2>\s\[AAMS\sID:\s[A-Z0-9]+\])?         # AAMS ID: in .it HH's
          \s-\s
          (?P<DATETIME>.*$)
        """ % substitutions, re.MULTILINE|re.VERBOSE)

m = re_GameInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,HID,TOUR,TOURNO,TOURNAME,GAME,LIMIT,SHOOTOUT,LEVEL,CURRENCY,SB,BB,BUB,CAP,CAPAMT,ISO,BLAH2,DATETIME
0,340571453,None,None,None,PLO,None,None,None,$,0.02,0.05,None,None,None,None,None,2023/01/03 11:24:11


In [4]:
re_Action           = re.compile(r"""
                        ^%(PLYR)s:(?P<ATYPE>\sbets|\schecks|\sraises|\scalls|\sfolds|\sdiscards|\sstands\spat)
                        (\s%(CUR)s(?P<BET>[,.\d]+))?(\sto\s%(CUR)s(?P<BETTO>[,.\d]+))?  # the number discarded goes in <BET>
                        \s*(and\sis\sall.in)?
                        (and\shas\sreached\sthe\s[%(CUR)s\d\.,]+\scap)?
                        (\son|\scards?)?
                        (\s\(disconnect\))?
                        (\s\[(?P<CARDS>.+?)\])?\s*$"""
                         %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_Action.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,PNAME,ATYPE,BET,BETTO,CARDS
0,4bf5b1f5,calls,0.05,None,None


In [32]:
re_Action           = re.compile(r"""
                        ^%(PLYR)s:(?P<ATYPE>\sbets|\schecks|\sraises|\scalls|\sfolds|\sdiscards|\sstands\spat|\sChooses\sto\sEV\sCashout|\sReceives\sCashout)
                        (\s%(CUR)s(?P<BET>[,.\d]+))?(\sto\s%(CUR)s(?P<BETTO>[,.\d]+))?  # the number discarded goes in <BET>
                        \s*(and\sis\sall.in)?
                        (and\shas\sreached\sthe\s[%(CUR)s\d\.,]+\scap)?
                        (\son|\scards?)?
                        (\s\(disconnect\))?
                        (\s\[(?P<CARDS>.+?)\])?\s*$"""
                         %  substitutions, re.MULTILINE|re.VERBOSE)

m = re_Action.search("d38f8e13: Chooses to EV Cashout")
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,PNAME,ATYPE,BET,BETTO,CARDS
0,d38f8e13,Chooses to EV Cashout,None,None,None
